<img src=diagram.png style="float:right;width:300px;height:400px;">

# Wellhead Relief Valve Sizing
-----------
## Quistorff Water System
Well S05 - AAE039  
Eng File \#3825   
Author: Ryan Haseman  
Date: 5-20-2019 

**Purpose:** 
Hydro-Pnuematics are insufficient at this site. This site will be converted from H-PH to VFD controlled. Wellhead plumbing to be re-piped and relief valve to be replaced. The relief valve serves to prevent the well pump from "dead-heading".


In [1]:
from Water import Pipe, Pump, tools

In [2]:
# maximum daily demand
Q_normal = 80   # gpm
Q_shutoff = 13  # gpm
print('Normal Flow =', round(Q_normal,2), 'gpm', '    Minimum Flow =', round(Q_shutoff, 2), 'gpm')

Normal Flow = 80 gpm     Minimum Flow = 13 gpm


In [3]:
# elevations
SWL = 122   # ft from top of well casing
hiLot_elevation = 320     # ft
wellhead_elevation = 287  # ft
valve_height = 1.5       # ft

# desired pressure at highest lot
hiLot_pressure = tools.psi2ft(psi=45)   # ft of head

In [4]:
specific_capacity = 0.2986     # ft/gpm
PWL_norm = SWL + Q_normal * specific_capacity   # ft from top of well casing
PWL_shutoff = SWL + Q_shutoff * specific_capacity # at shutoff conditions
print('PWL under normal conditions =', round(PWL_norm,2), 'ft bgs', '     PWL under shutoff conditions =', round(PWL_shutoff,2), 'ft bgs')

PWL under normal conditions = 145.89 ft bgs      PWL under shutoff conditions = 125.88 ft bgs


### Losses

In [6]:
riser_length = 168  # ft
suction_depth = 172 # ft

# pump to wellhead
riser_length = 168  # ft
pipe_pmp2wh = Pipe(length=riser_length, size=2, kind='GALVANIZED PIPE', sch=40)
pipe_pmp2wh.fitting('valve', 'column_check', qty=1, Kvalue=3)

# wellhead to distribution
pipe_wh2dist = Pipe(length=108, size=3, kind='GALVANIZED PIPE', sch=40)
pipe_wh2dist.fitting('tee_branch', 'standard_threaded', qty=2)
pipe_wh2dist.fitting('elbow_90', 'standard_threaded', qty=1)
pipe_wh2dist.fitting('flowmeter', 'turbine', qty=1, Kvalue=.1)
pipe_wh2dist.fitting('tee_through', 'standard_threaded', qty=3)
pipe_wh2dist.fitting('valve', 'gate', qty=1)

# distribution to water main
pipe_dist2ml = Pipe(length=200, size=2.5, kind='PVC', sch=40)
pipe_dist2ml.fitting('tee_branch', 'standard_glued', qty=1)
pipe_dist2ml.fitting('elbow_90', 'standard_glued', qty=1)
pipe_dist2ml.fitting('valve', 'gate', qty=1)

# mainline to hilot
pipe_ml2hilot = Pipe(length=2500, size=8, kind='PVC', sch='C900 DR-18')
pipe_ml2hilot.fitting('valve', 'gate', qty=1)

In [7]:
# calculating discharge side losses
losses_norm = pipe_pmp2wh.get_losses(flow=Q_normal) +\
              pipe_wh2dist.get_losses(flow=Q_normal) +\
              pipe_dist2ml.get_losses(flow=Q_normal) +\
              pipe_ml2hilot.get_losses(flow=Q_normal)

losses_shutoff = pipe_pmp2wh.get_losses(flow=Q_shutoff)

# print result
result = 'Discharge Losses under normal conditions: {:.2f} ft'.format(losses_norm)
result2 = 'Discharge Losses under shutoff conditions: {:.2f} ft'.format(losses_shutoff)
print(result, '      ',result2)

Discharge Losses under normal conditions: 40.53 ft        Discharge Losses under shutoff conditions: 0.91 ft


In [11]:
# pressure at pump suction
discharge_head = hiLot_pressure +\
                 hiLot_elevation +\
                 PWL_norm +\
                 losses_norm -\
                 wellhead_elevation
pressure_norm = tools.ft2psi(discharge_head)

# shutoff_head relative to valve
shutoff_head = 650  # ft (from pump curve)
head_at_minflow = 620  # ft
high_head = head_at_minflow - (suction_depth - PWL_shutoff) - losses_shutoff
relief_pressure = tools.ft2psi(high_head)

print('relief pressure at shutoff conditions =',round(relief_pressure,2), 'psi')
print('normal operating well head pressure =',round(pressure_norm,2), 'psi')

relief pressure at shutoff conditions = 248.29 psi
normal operating well head pressure = 140.08 psi
